# libraries


In [9]:
     
import serial
import time
import cv2
import numpy as np

# modbus to regular


In [ ]:
import struct
from pymodbus.client import ModbusSerialClient as ModbusClient

# הגדרות החיבור
PORT = "COM11"  # שם הפורט
BAUDRATE = 230400  # בודרייט אופייני ל-MODBUS
TIMEOUT = 2  # זמן מקסימלי להמתנה לתשובה

# יצירת לקוח Modbus RTU
client = ModbusClient(
    port=PORT,
    baudrate=BAUDRATE,
    timeout=TIMEOUT,
    parity='N',
    stopbits=1,
    bytesize=8
)


def read_modbus_registers(slave_id, register_address, count=1):
    """
    קורא רגיסטרים בפרוטוקול MODBUS RTU ומחזיר את הנתונים כערכי A2D.
    """
    if not client.connect():
        print("Failed to connect to Modbus device.")
        return None

    try:
        response = client.read_input_registers(address=register_address, count=count, slave=slave_id)
        if response.isError():
            print("Error reading Modbus registers")
            return None
        return response.registers
    except Exception as e:
        print(f"Error: {e}")
        return None
    finally:
        client.close()

def convert_to_a2d(modbus_values, v_ref=5.0, resolution=16):
    """
    ממיר ערכים שהתקבלו ממודבוס לערכים אנלוגיים (A2D).
    הנחת יסוד: ערכי 16 ביט.
    """
    max_value = 2**resolution - 1  # 65535 עבור 16 ביט
    return [(val / max_value) * v_ref for val in modbus_values]

if __name__ == "__main__":
    slave_id = 1
    register_address = 0

    print("🔄 Attempting connection...")
    modbus_data = read_modbus_registers(slave_id, register_address, count=2)

    if modbus_data:
        print(f"✅ Raw Modbus Data: {modbus_data}")
        a2d_values = convert_to_a2d(modbus_data)
        print(f"📊 Converted A2D Values (V): {a2d_values}")
    else:
        print("⚠️ Failed to read data. Check connection and parameters.")

# main - GAS

In [ ]:
import serial
import time
import cv2
import numpy as np
from threading import Thread

# הגדרות תצוגה
width, height = 500, 500
font = cv2.FONT_HERSHEY_SIMPLEX
sampels_per_sec=100
time_interval=5 #sec
sampels_num=sampels_per_sec*time_interval
c3=[]
channel_values = {1: [], 2: [], 3: [], 4: []}
# לאחר חישוב ממוצעים – כאן יאוחסנו הממוצעים של כל ערוץ
last_computed_averages = None
# משתנה לשליטה שמונע הפעלת חישוב במקביל
averaging_in_progress = False

def process_channel(channel_bytes):

    swapped_bytes = channel_bytes[2:3] + channel_bytes[1:2] + channel_bytes[0:1]
    value = int.from_bytes(swapped_bytes, byteorder='big')
    return value

def process_packet(packet):

    data_bytes = packet[4:]
    channels = [data_bytes[i:i+3] for i in range(0, 12, 3)]
    return [process_channel(ch) for ch in channels]

def compute_averages_for_interval():
    """
    לאחר לחיצה על 'a', פונקציה זו תרוץ במשך 2 שניות.
    כל 0.5 שניות היא בודקת את הדגימות שנאספו באותו האינטרוול,
    לוקחת את הערך המקסימלי לכל ערוץ, ושומרת אותם ברשימה מקומית.
    בסיום 4 אינטרוולים (2 שניות), היא מחשבת ממוצע של הערכים המקסימליים לכל ערוץ
    ומעדכנת את המשתנה הגלובלי last_computed_averages.
    """
    global last_computed_averages, averaging_in_progress, channel_values

    local_max = {1: [], 2: [], 3: [], 4: []}
    intervals_collected = 0

    while intervals_collected < 4:
        time.sleep(1)  # מחכים 0.5 שניות לאיסוף דגימות
        # לכל ערוץ, מחשבים את המקסימום שהצטבר באותו האינטרוול
        for ch_num in range(1, 5):
            if channel_values[ch_num]:
                local_max[ch_num].append(max(channel_values[ch_num]))
            #else:
                #   local_max[ch_num].append(0)
        print(channel_values[3])
        # מנקים את הרשימות כדי להתחיל אינטרוול חדש
        for ch in channel_values.values():
            ch.clear()
        intervals_collected += 1

    # חישוב ממוצע של 4 הערכים המקסימליים לכל ערוץ
    averages = {}
    for ch_num in range(1, 5):
        if local_max[ch_num]:
            averages[ch_num] = sum(local_max[ch_num]) / len(local_max[ch_num])
        else:
            averages[ch_num] = 0

    last_computed_averages = averages
    averaging_in_progress = False


def compute_averages_for_interval2(c3):


    local_max=max(c3)
    return local_max




def read_serial(port='COM11', baudrate=230400):
    global last_computed_averages, averaging_in_progress
    ser = serial.Serial(port, baudrate, timeout=0.01)  # הפחתת זמן ה-Timeout לשיפור מהירות הקריאה
    print(f"Connected to {port} at {baudrate} baud.")

    try:
        while True:
            black_image = np.zeros((height, width, 3), np.uint8)
            
            # קריאת כל הנתונים הזמינים במכה אחת במקום בית אחד כל פעם
            bytes_available = ser.in_waiting
            if bytes_available >= 16:
                data = ser.read(bytes_available)
                
                # חיפוש חבילות שמתחילות ב־0xFF
                packets = [data[i:i+16] for i in range(len(data)-15) if data[i] == 0xFF]
                
                for packet in packets:
                    channels = process_packet(packet)
                    
                    # סינון לפי ערך סף
                    if channels[2] > 20000:
                        channel_values[3].append(channels[2])
                        cv2.putText(black_image, f'Channel 3: {channels[2]}', (50, 100), font, 1, (255, 0, 0), 2)  # הטקסט הקדמי
                
                # הצגת הממוצעים אם קיימים
                if last_computed_averages:
                    y_pos = 200
                    for ch_num, avg in last_computed_averages.items():
                        cv2.putText(black_image, f'Ch {ch_num} Avg Max: {avg:.2f}', (50, y_pos), font, 0.8, (0, 255, 0), 2)
                        y_pos += 40

                cv2.imshow('Channel Data', black_image)
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('a') and not averaging_in_progress:
                    averaging_in_progress = True
                    last_computed_averages = None
                    Thread(target=compute_averages_for_interval, daemon=True).start()

    except KeyboardInterrupt:
        print("Stopped by user.")
    finally:
        ser.close()
        cv2.destroyAllWindows()

# קריאה לפונקציה הראשית
read_serial(port='COM5', baudrate=230400)

Connected to COM5 at 230400 baud.


# tests


In [ ]:
# Optical Bench Alignment Testing System
import serial
import time
import wave
import cv2
import numpy as np
import pandas as pd
from threading import Thread
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from datetime import datetime

# הגדרות תצוגה
width, height = 600, 700
font = cv2.FONT_HERSHEY_SIMPLEX

def process_channel(channel_bytes):
    swapped_bytes = channel_bytes[2:3] + channel_bytes[1:2] + channel_bytes[0:1]
    value = int.from_bytes(swapped_bytes, byteorder='big')
    return value

def process_packet(packet):
    data_bytes = packet[4:]
    channels = [data_bytes[i:i+3] for i in range(0, 12, 3)]
    return [process_channel(ch) for ch in channels]

def update_live_plot(angles, ch1_norm, ch2_norm, ratio_values, selected_channels, test_mode, current_axis, fig=None):
    # Use the existing figure or create one if it doesn't exist
    if fig is None:
        fig = plt.figure(figsize=(12, 8))
    
    # Clear the figure completely
    fig.clear()
    
    if test_mode == "single":
        ax1 = fig.add_subplot(2, 1, 1)
        if len(ch1_norm) > 0:
            ax1.plot(angles, ch1_norm, 'o-', label=f'Ch{selected_channels[0]} (normalized)', color='blue')
        if len(ch2_norm) > 0:
            ax1.plot(angles, ch2_norm, 'o-', label=f'Ch{selected_channels[1]} (normalized)', color='red')
        ax1.set_xlabel('Angle (degrees)')
        ax1.set_ylabel('Normalized Signal')
        ax1.set_title('Channel Alignment Test')
        ax1.legend()
        ax1.grid(True)
        
        ax2 = fig.add_subplot(2, 1, 2)
        if len(ratio_values) > 0:
            ax2.plot(angles, ratio_values, 'o-', label=f'Ratio Ch{selected_channels[0]}/Ch{selected_channels[1]}', color='green')
        ax2.set_xlabel('Angle (degrees)')
        ax2.set_ylabel('Ratio')
        ax2.set_title('Channel Ratio')
        ax2.legend()
        ax2.grid(True)
    
    else:  # dual axis
        ax1 = fig.add_subplot(2, 1, 1)
        x_indices = [i for i, axis in enumerate(current_axis) if axis == 'X']
        y_indices = [i for i, axis in enumerate(current_axis) if axis == 'Y']
        
        if x_indices:
            x_angles = [angles[i] for i in x_indices]
            x_ch1 = [ch1_norm[i] for i in x_indices]
            x_ch2 = [ch2_norm[i] for i in x_indices]
            ax1.plot(x_angles, x_ch1, 'o-', label=f'X-axis Ch{selected_channels[0]}', color='blue')
            ax1.plot(x_angles, x_ch2, 'o-', label=f'X-axis Ch{selected_channels[1]}', color='red')
        
        if y_indices:
            y_angles = [angles[i] for i in y_indices]
            y_ch1 = [ch1_norm[i] for i in y_indices]
            y_ch2 = [ch2_norm[i] for i in y_indices]
            ax1.plot(y_angles, y_ch1, 's-', label=f'Y-axis Ch{selected_channels[0]}', color='lightblue')
            ax1.plot(y_angles, y_ch2, 's-', label=f'Y-axis Ch{selected_channels[1]}', color='lightcoral')
        
        ax1.set_xlabel('Angle (degrees)')
        ax1.set_ylabel('Normalized Signal')
        ax1.set_title('Dual Axis Alignment Test')
        ax1.legend()
        ax1.grid(True)
        
        ax2 = fig.add_subplot(2, 1, 2)
        if x_indices:
            x_ratio = [ratio_values[i] for i in x_indices]
            ax2.plot(x_angles, x_ratio, 'o-', label='X-axis Ratio', color='green')
        if y_indices:
            y_ratio = [ratio_values[i] for i in y_indices]
            ax2.plot(y_angles, y_ratio, 's-', label='Y-axis Ratio', color='orange')
        
        ax2.set_xlabel('Angle (degrees)')
        ax2.set_ylabel('Ratio')
        ax2.set_title('Channel Ratios')
        ax2.legend()
        ax2.grid(True)
    
    fig.tight_layout()
    plt.draw()
    plt.pause(0.1)
    
    return fig

def read_serial_alignment_test(port='COM11', baudrate=230400):
    sampels_per_sec = 500
    time_interval = 3  # sec
    sampels_num = sampels_per_sec * time_interval
    
    # Get test configuration from user
    print("=== Optical Bench Alignment Test Setup ===")
    channels_input = input("Enter channels to monitor (e.g., 1,3): ")
    selected_channels = [int(x.strip()) for x in channels_input.split(',')]
    
    test_mode = input("Test mode - single axis (s) or dual axis (d)? ").lower()
    test_mode = "single" if test_mode == "s" else "dual"
    
    print("\n=== Setup Complete ===")
    print("1. Move to maximum power position for both X and Y")
    print("2. Press 'c' for calibration")
    print("3. Press 'g' to set starting angle")
    print("4. Move to starting position for first axis")
    print("5. Press 'a' to record measurements (angle auto-increments)")
    print("6. Press 's' to save and continue/finish")
    print("7. Press 'q' to quit")
    
    # Initialize variables
    c1, c2, c3, c4 = [], [], [], [], 
    ch_values = {1: [], 2: [], 3: [], 4: []}
    c1_med = c2_med = c3_med = c4_med = 0
    c1_current = c2_current = c3_current = c4_current = 0  # Real-time display values
    
    # Test data storage
    angles = []
    measurements = {1: [], 2: [], 3: [], 4: []}
    ch1_norm = []
    ch2_norm = []
    ratio_values = []
    current_axis = []
    
    # Calibration values
    cal_ch1 = cal_ch2 = 1
    is_calibrated = False
    
    # Current measurement tracking
    measurement_count = 0
    current_test_axis = "X"
    current_angle = None  # Will be set when user presses 'g'
    angle_set = False     # Track if angle has been set
    
    # Initialize matplotlib
    plt.ion()
    plot_fig = plt.figure(figsize=(12, 8))
    plot_fig.show()
    
    ser = serial.Serial(port, baudrate, timeout=0.01)
    print(f"Connected to {port} at {baudrate} baud.")
    
    try:
        while True:
            black_image = np.zeros((height, width, 3), np.uint8)
            
            bytes_available = ser.in_waiting
            if bytes_available >= 16:
                data = ser.read(bytes_available)
                packets = [data[i:i+16] for i in range(len(data)-15) if data[i] == 0xFF]
                
                for packet in packets:
                    channels = process_packet(packet)
                    c1.append(channels[0])
                    c2.append(channels[1])
                    c3.append(channels[2])
                    c4.append(channels[3])
                    
                    # Update real-time display values immediately
                    c1_current = channels[0]
                    c2_current = channels[1] 
                    c3_current = channels[2]
                    c4_current = channels[3]
                    
                    if len(c3) > sampels_num:
                        c1, c2, c3, c4 = map(np.array, [c1, c2, c3, c4])
                        
                        # Calculate medians
                        c1_med = np.median(c1[c1 > 70000]) if np.any(c1 > 70000) else 0
                        c2_med = np.median(c2[c2 > 70000]) if np.any(c2 > 70000) else 0
                        c3_med = np.median(c3[c3 > 70000]) if np.any(c3 > 70000) else 0
                        c4_med = np.median(c4[c4 > 70000]) if np.any(c4 > 70000) else 0
                        
                        # Cap extreme values
                        if c1_med > 8000000: c1_med = 0
                        if c2_med > 8000000: c2_med = 0
                        if c3_med > 8000000: c3_med = 0
                        if c4_med > 8000000: c4_med = 0
                        
                        c1, c2, c3, c4 = [], [], [], []
            
            # Create display
            black_image = np.zeros((height, width, 3), np.uint8)
            cv2.putText(black_image, f'Ch1: {np.round(c1_current/1e6,2)}M', (50, 50), font, 0.8, (0, 255, 0), 2)
            cv2.putText(black_image, f'Ch2: {np.round(c2_current/1e6,2)}M', (50, 90), font, 0.8, (0, 255, 0), 2)
            cv2.putText(black_image, f'Ch3: {np.round(c3_current/1e6,2)}M', (50, 130), font, 0.8, (0, 255, 0), 2)
            cv2.putText(black_image, f'Ch4: {np.round(c4_current/1e6,2)}M', (50, 170), font, 0.8, (0, 255, 0), 2)
            
            # Test info
            cv2.putText(black_image, f'Test: Ch{selected_channels[0]},Ch{selected_channels[1]}', (50, 220), font, 0.7, (255, 255, 0), 2)
            cv2.putText(black_image, f'Mode: {test_mode}', (50, 250), font, 0.7, (255, 255, 0), 2)
            cv2.putText(black_image, f'Measurements: {measurement_count}', (50, 280), font, 0.7, (255, 255, 0), 2)
            if current_angle is not None:
                cv2.putText(black_image, f'Current angle: {current_angle}', (50, 310), font, 0.7, (255, 255, 0), 2)
            if test_mode == "dual":
                cv2.putText(black_image, f'Current axis: {current_test_axis}', (50, 340), font, 0.7, (255, 255, 0), 2)
            
            # Calibration status
            cal_status = "CALIBRATED" if is_calibrated else "NOT CALIBRATED"
            cal_color = (0, 255, 0) if is_calibrated else (0, 0, 255)
            cv2.putText(black_image, cal_status, (50, 380), font, 0.7, cal_color, 2)
            
            # Instructions
            cv2.putText(black_image, "Keys:", (50, 420), font, 0.6, (255, 255, 255), 2)
            cv2.putText(black_image, "c - Calibrate", (50, 445), font, 0.5, (255, 255, 255), 1)
            cv2.putText(black_image, "g - Set starting angle", (50, 465), font, 0.5, (255, 255, 255), 1)
            cv2.putText(black_image, "a - Add measurement", (50, 485), font, 0.5, (255, 255, 255), 1)
            cv2.putText(black_image, "s - Save and continue", (50, 505), font, 0.5, (255, 255, 255), 1)
            cv2.putText(black_image, "q - Quit", (50, 525), font, 0.5, (255, 255, 255), 1)
            
            cv2.imshow('Optical Bench Alignment Test', black_image)
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('q'):
                break
                
            elif key == ord('c'):
                # Calibration mode
                ch1_val = [c1_med, c2_med, c3_med, c4_med][selected_channels[0]-1]
                ch2_val = [c1_med, c2_med, c3_med, c4_med][selected_channels[1]-1]
                
                if ch1_val > 0 and ch2_val > 0:
                    cal_ch1 = ch1_val
                    cal_ch2 = ch2_val
                    is_calibrated = True
                    print(f"Calibrated! Ch{selected_channels[0]}: {cal_ch1/1e6:.2f}M, Ch{selected_channels[1]}: {cal_ch2/1e6:.2f}M")
                else:
                    print("Cannot calibrate - one or both channels have zero values")
                    
            elif key == ord('g'):
                # Set starting angle
                print(f"\n--- Set Starting Angle ---")
                if test_mode == "dual":
                    print(f"Current axis: {current_test_axis}")
                
                try:
                    current_angle = float(input("Enter starting angle (degrees): "))
                    angle_set = True
                    print(f"Starting angle set to {current_angle}°. Press 'a' to add measurements (angle will auto-increment).")
                except ValueError:
                    print("Invalid angle entered. Please try again.")
                    
            elif key == ord('a'):
                # Add measurement
                if is_calibrated:
                    if not angle_set:
                        print("Please set starting angle first (press 'g')")
                        continue
                    
                    ch1_val = [c1_med, c2_med, c3_med, c4_med][selected_channels[0]-1]
                    ch2_val = [c1_med, c2_med, c3_med, c4_med][selected_channels[1]-1]
                    
                    # Store raw measurements
                    angles.append(current_angle)
                    measurements[1].append(c1_med)
                    measurements[2].append(c2_med)
                    measurements[3].append(c3_med)
                    measurements[4].append(c4_med)
                    
                    # Calculate normalized values
                    norm_ch1 = (ch1_val / cal_ch1) if cal_ch1 > 0 else 0
                    norm_ch2 = (ch2_val / cal_ch2) if cal_ch2 > 0 else 0
                    ch1_norm.append(norm_ch1)
                    ch2_norm.append(norm_ch2)
                    
                    # Calculate ratio
                    ratio = (norm_ch1 / norm_ch2) if norm_ch2 > 0 else 0
                    ratio_values.append(ratio)
                    
                    # Track axis for dual mode
                    current_axis.append(current_test_axis)
                    
                    measurement_count += 1
                    print(f"Measurement {measurement_count}: Angle {current_angle}°, Ch{selected_channels[0]} norm: {norm_ch1:.3f}, Ch{selected_channels[1]} norm: {norm_ch2:.3f}, Ratio: {ratio:.3f}")
                    
                    # Update live plot
                    plot_fig = update_live_plot(angles, ch1_norm, ch2_norm, ratio_values, selected_channels, test_mode, current_axis, plot_fig)
                    
                    # Auto-increment angle for next measurement
                    current_angle += 1
                else:
                    print("Please calibrate first (press 'c' at maximum power position)")
                    
            elif key == ord('s'):
                # Save measurements
                if len(angles) > 0:
                    if test_mode == "single":
                        # Single axis - save and finish
                        filename = input("Enter filename (without extension): ")
                        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                        save_path = rf"C:\Users\LAB3\Downloads\ir_bench\{filename}_{timestamp}.xlsx"
                        
                        # Create DataFrame
                        df_data = {
                            'angle': angles,
                            f'ch{selected_channels[0]}': [measurements[selected_channels[0]][i] for i in range(len(angles))],
                            f'ch{selected_channels[1]}': [measurements[selected_channels[1]][i] for i in range(len(angles))],
                            f'ch{selected_channels[0]}_normalized': ch1_norm,
                            f'ch{selected_channels[1]}_normalized': ch2_norm,
                            f'ratio_{selected_channels[0]}_{selected_channels[1]}': ratio_values
                        }
                        
                        df = pd.DataFrame(df_data)
                        df.to_excel(save_path, index=False)
                        print(f"Data saved to {save_path}")
                        
                        # Show final plot
                        plt.ioff()
                        update_live_plot(angles, ch1_norm, ch2_norm, ratio_values, selected_channels, test_mode, current_axis)
                        plt.show()
                        break
                        
                    else:
                        # Dual axis mode
                        if current_test_axis == "X":
                            # Finished X axis, start Y axis
                            print("X-axis measurements complete. Move to starting position for Y-axis test.")
                            input("Press Enter when ready to start Y-axis measurements...")
                            current_test_axis = "Y"
                            # Reset angle settings for Y-axis
                            current_angle = None
                            angle_set = False
                            print("Now set starting angle for Y-axis (press 'g'), then collect measurements with 'a'.")
                        else:
                            # Finished both axes - save and exit
                            filename = input("Enter filename (without extension): ")
                            timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                            save_path = rf"C:\Users\LAB3\Downloads\ir_bench\{filename}_dual_axis_{timestamp}.xlsx"
                            
                            # Create DataFrame with axis information
                            df_data = {
                                'angle': angles,
                                'axis': current_axis,
                                f'ch{selected_channels[0]}': [measurements[selected_channels[0]][i] for i in range(len(angles))],
                                f'ch{selected_channels[1]}': [measurements[selected_channels[1]][i] for i in range(len(angles))],
                                f'ch{selected_channels[0]}_normalized': ch1_norm,
                                f'ch{selected_channels[1]}_normalized': ch2_norm,
                                f'ratio_{selected_channels[0]}_{selected_channels[1]}': ratio_values
                            }
                            
                            df = pd.DataFrame(df_data)
                            df.to_excel(save_path, index=False)
                            print(f"Data saved to {save_path}")
                            
                            # Show final plot
                            plt.ioff()
                            update_live_plot(angles, ch1_norm, ch2_norm, ratio_values, selected_channels, test_mode, current_axis)
                            plt.show()
                            break
                else:
                    print("No measurements to save!")
    
    except KeyboardInterrupt:
        print("Stopped by user.")
    finally:
        ser.close()
        cv2.destroyAllWindows()
        plt.close('all')

# Start the alignment test
read_serial_alignment_test(port='COM3', baudrate=230400)


Connected to COM5 at 230400 baud.


# main - FLAME

In [ ]:
#only print
import serial

port = 'COM16'
baudrate = 230400  

try:
    ser = serial.Serial(port, baudrate, timeout=1)
    print(f"✅ Connected to {ser.name}")
except serial.SerialException as e:
    print(f"❌ Failed to connect: {e}")
    exit()

def process_packet(packet):
    if packet.startswith(b'\xFF\x01\x09'):  # בדיקה אם זה פאקט תקין
        packet = packet[6:]  # מוחק את 6 הבתים הראשונים
        sensors = [packet[i:i+3] for i in range(0, 18, 3)]  # מחלק ל-6 רגשים (כל אחד 3 בתים)

        processed_sensors = []
        for i in range(3, 6):  # לוקח רק את רגשים 4, 5, 6
            sensor = sensors[i]
            reordered = sensor[2:3] + sensor[1:2] + sensor[0:1]  # מחליף סדר
            value = int.from_bytes(reordered, byteorder='big')  # ממיר למספר שלם
            processed_sensors.append(value)

        print(f"📊 Sensor Values (4,5,6): {processed_sensors}")  # מציג רק 3 ערכים רצויים

try:
    buffer = b""
    while True:
        if ser.in_waiting > 0:
            buffer += ser.read(24)  # קורא 24 בתים (גודל פאקט מלא)
            if len(buffer) >= 24:
                process_packet(buffer[:24])  # שולח לעיבוד
                buffer = buffer[24:]  # מנקה את החלק המעובד
except KeyboardInterrupt:
    print("\n🔌 Disconnected.")
    ser.close()


✅ Connected to COM16
📊 Sensor Values (4,5,6): [8524, 2450, 4372]
📊 Sensor Values (4,5,6): [8519, 2454, 4388]
📊 Sensor Values (4,5,6): [8519, 2449, 4370]
📊 Sensor Values (4,5,6): [8539, 2480, 4405]
📊 Sensor Values (4,5,6): [8530, 2460, 4389]
📊 Sensor Values (4,5,6): [8541, 2469, 4406]
📊 Sensor Values (4,5,6): [8542, 2470, 4396]
📊 Sensor Values (4,5,6): [8523, 2456, 4382]
📊 Sensor Values (4,5,6): [8546, 2468, 4395]
📊 Sensor Values (4,5,6): [8558, 2496, 4416]
📊 Sensor Values (4,5,6): [8538, 2474, 4406]
📊 Sensor Values (4,5,6): [8550, 2462, 4400]
📊 Sensor Values (4,5,6): [8559, 2467, 4387]
📊 Sensor Values (4,5,6): [8553, 2477, 4406]
📊 Sensor Values (4,5,6): [8555, 2474, 4417]
📊 Sensor Values (4,5,6): [8571, 2498, 4419]
📊 Sensor Values (4,5,6): [8552, 2468, 4404]
📊 Sensor Values (4,5,6): [8574, 2482, 4429]
📊 Sensor Values (4,5,6): [8556, 2458, 4399]
📊 Sensor Values (4,5,6): [8558, 2479, 4404]
📊 Sensor Values (4,5,6): [8558, 2480, 4402]
📊 Sensor Values (4,5,6): [8532, 2465, 4379]
📊 Sensor Va

In [ ]:
#image test

import serial
import cv2
import numpy as np
import threading

port = 'COM16'
baudrate = 230400  

# הגדרות תמונה
width, height = 500, 500
font = cv2.FONT_HERSHEY_SIMPLEX

# משתנה לשמירת הערכים האחרונים
latest_values = [0, 0, 0]
stop_thread = False  # משתנה לסגירת התהליך

try:
    ser = serial.Serial(port, baudrate, timeout=0.01)  # הפחתת זמן Timeout לקריאה מהירה יותר
    print(f"✅ Connected to {ser.name}")
except serial.SerialException as e:
    print(f"❌ Failed to connect: {e}")
    exit()

def process_packet(packet):
    global latest_values
    if packet.startswith(b'\xFF\x01\x09'):  # בדיקה אם זה פאקט תקין
        packet = packet[6:]  # מוחק את 6 הבתים הראשונים
        sensors = [packet[i:i+3] for i in range(0, 18, 3)]  # מחלק ל-6 רגשים (כל אחד 3 בתים)

        processed_sensors = []
        for i in range(3, 6):  # לוקח רק את רגשים 4, 5, 6
            sensor = sensors[i]
            reordered = sensor[2:3] + sensor[1:2] + sensor[0:1]  # מחליף סדר
            value = int.from_bytes(reordered, byteorder='big')  # ממיר למספר שלם
            processed_sensors.append(value)

        latest_values = processed_sensors  # מעדכן את הערכים האחרונים

# **פונקציה שמעדכנת את התצוגה במהירות גבוהה יותר**
def display_loop():
    global stop_thread
    while not stop_thread:
        # יצירת תמונה שחורה
        black_image = np.zeros((height, width, 3), np.uint8)

        # הצגת הנתונים על התמונה
        for idx, value in enumerate(latest_values):
            text = f"Sensor {idx+4}: {value}"  # רגשים 4, 5, 6
            cv2.putText(black_image, text, (50, 150 + idx * 50), font, 1, (0, 255, 0), 2)

        # הצגת התמונה
        cv2.imshow("Sensor Data", black_image)
        key = cv2.waitKey(1) & 0xFF  # מרענן את ה-GUI

        if key == ord('q'):  # אפשרות לסגור את החלון
            print("\n🔌 Disconnected.")
            stop_thread = True
            ser.close()
            cv2.destroyAllWindows()
            exit()

        # השהיה קצרה מאוד, שומר על רענון מהיר
        cv2.waitKey(1)

# **הפעלת Thread נפרד להצגת התמונה במהירות גבוהה יותר**
thread = threading.Thread(target=display_loop, daemon=True)
thread.start()

try:
    buffer = b""
    while True:
        if ser.in_waiting > 0:
            buffer += ser.read(24)  # קורא 24 בתים (גודל פאקט מלא)
            if len(buffer) >= 24:
                process_packet(buffer[:24])  # שולח לעיבוד
                buffer = buffer[24:]  # מנקה את החלק המעובד
except KeyboardInterrupt:
    print("\n🔌 Disconnected.")
    stop_thread = True
    ser.close()
    cv2.destroyAllWindows()


✅ Connected to COM16
